# Quick Start Example (Optional)

This notebook demonstrates basic data access patterns using DuckDB and pandas. For a deeper dive into building full analytics agents on MotherDuck (prompt design, MCP integration, security), see [MotherDuck’s analytics agent guide](https://motherduck.com/docs/key-tasks/ai-and-motherduck/building-analytics-agents/).

## Contents
- Tool installation
- Loading Parquet files
- SQL queries
- Log file parsing
- Submission format


## Install Tools

Run this once:


In [1]:
%pip install -q duckdb pandas pyarrow lancedb

import duckdb
import pandas as pd
from pathlib import Path

print("Ready.")


Note: you may need to restart the kernel to use updated packages.
Ready.


## Connect to DuckDB

DuckDB lets you run SQL queries on Parquet files. It's fast and works well for analytics.


In [2]:
con = duckdb.connect('retail.duckdb')
print("Connected to DuckDB")


Connected to DuckDB


## Load Data

Load Parquet files into DuckDB so you can query them:


In [3]:
# Load a single example table
# Update the path below if you place the file somewhere else

customer_path = Path('../dataset/parquets/customer.parquet').resolve()
if customer_path.exists():
    con.execute(f"CREATE OR REPLACE TABLE customer AS SELECT * FROM '{customer_path.as_posix()}'")
    
    print("Customer table loaded")
    print(con.execute("SHOW TABLES").df())
else:
    print("Customer Parquet not found. Copy it to dataset/parquets/customer.parquet first.")


Customer table loaded
       name
0  customer


## Run a Query

Example: Find top customers by spending


In [4]:
if customer_path.exists():
    query = """
    SELECT 
        c_customer_sk,
        c_first_name,
        c_last_name
    FROM customer
    ORDER BY c_customer_sk
    LIMIT 5
    """
    
    result = con.execute(query).df()
    print(result)


   c_customer_sk c_first_name c_last_name
0              1        Julie      Elkins
1              2       Samuel     Simmons
2              3     Franklin      Amador
3              4     Kathrine      Tuttle
4              5        James     Vincent


## Read Log Files

Logs are in JSONL format (one JSON object per line). You can use pandas:


In [5]:
if Path('dataset/logs/clickstream.jsonl').exists():
    logs = pd.read_json('dataset/logs/clickstream.jsonl', lines=True)
    print(f"Loaded {len(logs)} events")
    print(logs.head())


## Alternative: Query Logs with DuckDB

You can also load logs into DuckDB and use SQL:


In [6]:
if Path('dataset/logs/clickstream.jsonl').exists():
    con.execute("CREATE OR REPLACE TABLE logs AS SELECT * FROM read_json_auto('dataset/logs/clickstream.jsonl')")
    
    result = con.execute("""
        SELECT event_type, COUNT(*) as count 
        FROM logs 
        GROUP BY event_type
    """).df()
    
    print(result)


## Using LanceDB for Semantic Search

LanceDB is useful when you need to search by meaning rather than exact matches. Example use case: searching through log descriptions or PDF content.


In [ ]:
import lancedb

# Connect to LanceDB
db = lancedb.connect('lance_db')

# Example: Load log data into LanceDB
log_path = Path('../dataset/logs/clickstream.jsonl')
if log_path.exists():
    log_data = pd.read_json(log_path, lines=True)
    
    # Create a table
    table = db.create_table("logs", data=log_data, mode="overwrite")
    
    print(f"Loaded {len(log_data)} rows into LanceDB")
    
    # Query example: Filter logs
    results = table.search().where("event_type = 'product_view'").limit(5).to_pandas()
    print("\nSample product view events:")
    print(results)
    
    # Query LanceDB table with DuckDB
    arrow_table = table.to_lance()
    duckdb_results = con.query("SELECT * FROM arrow_table LIMIT 5").df()
    print("\nQuerying LanceDB table with DuckDB:")
    print(duckdb_results)
else:
    print("Log files not found. Make sure you've run the download_dataset.sh script.")


/Users/panthbhavsar/Documents/theory/modeler-hackathon-starter/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Log files not found.


NameError: name 'table' is not defined

## Submission Format

Save your answers in CSV format:

```csv
question_id,answer_type,answer_value,confidence,explanation
1,customer_id,12345,high,Top customer by revenue
1,total_spent,50000,high,Sum of net_paid
```

Create submissions programmatically:


In [ ]:
submission = pd.DataFrame([
    {'question_id': 1, 'answer_type': 'customer_id', 'answer_value': 12345, 'confidence': 'high', 'explanation': 'Top customer by revenue'},
    {'question_id': 1, 'answer_type': 'total_spent', 'answer_value': 50000, 'confidence': 'high', 'explanation': 'Sum of net_paid'},
])

print(submission)

# To save:
# submission.to_csv('my_submission.csv', index=False)


## Competition Structure

**Training Round (12:30-2:00):** 25 questions with answers provided. Practice only, no submission required.

**Test Round (2:00-6:00):** 30 questions without answers. Submit by 6:00 PM. Worth 70% of final score.

**Holdout Round (6:00-7:30):** 20 secret questions. We run your system automatically. Worth 30% of final score.

That's the basics. Check the README for more details.


## Tool Use Cases

**DuckDB:** SQL queries on structured data (Parquet tables) and logs. Fast for aggregations, joins, filtering.

**LanceDB:** Semantic search when you need to find things by meaning, not exact matches. Good for searching PDFs or finding similar log entries.

**MotherDuck:** Cloud version of DuckDB. Useful for sharing data/queries with teammates or working with larger datasets.


## MCP Server Quickstart

1. **Pick a language/runtime.** MCP servers only need stdin/stdout plus JSON-RPC. Python (`mcp`), TypeScript (`@modelcontextprotocol/server`), or Go all work.
2. **Choose resources.** Decide what data you’ll expose (DuckDB tables, Parquet files, PDF search). Give each a stable URI so clients know how to reference them.
3. **Implement tools.** Each MCP tool wraps an action—run SQL, summarize a log window, fetch a PDF section. Keep inputs/outputs typed and minimal so LLMs can call them safely.
4. **Advertise capabilities.** In `initialize` return your tool/resource metadata so Cursor/Claude Desktop lists them automatically.
5. **Run & register.** Start the server (e.g., `python mcp_server.py`) and add that command under `Cursor → Settings → MCP Servers`.

Minimal Python scaffold:

```python
from mcp.server.fastmcp import FastMCPServer
import duckdb

con = duckdb.connect('retail.duckdb')
server = FastMCPServer()

@server.tool()
def describe_customer(limit: int = 5) -> list[dict[str, str]]:
    """Return sample customer rows from DuckDB."""
    rows = con.execute(
        "SELECT c_customer_sk, c_first_name, c_last_name FROM customer LIMIT ?",
        [limit],
    ).fetchall()
    return [dict(row) for row in rows]

if __name__ == "__main__":
    server.run()
```

Register `python mcp_server.py` as a custom MCP server and the `describe_customer` tool becomes available directly in your prompts.

> **Shortcut:** Don’t want to build your own? MotherDuck ships an OSS MCP server that connects to both DuckDB and MotherDuck backends, complete with SaaS/read-only modes and Claude/Cursor examples. Install it via `uvx mcp-server-motherduck …` using the instructions in their repo: https://github.com/motherduckdb/mcp-server-motherduck.
